In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

#### Here Actual Output is wrong to test the model prediction answers.

In [5]:

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

# the following 2 hyperparameters are task-specific
max_source_length = 512
max_target_length = 128

# Suppose we have the following 2 training examples:
input_sequence_1 = "New Thing in Town"
output_sequence_1 = "Comment éliminer les données importantes?"

input_sequence_2 = "My name is moeed"
output_sequence_2 = "Mon nom est moeed"

# encode the inputs
task_prefix = "translate English to French: "
input_sequences = [input_sequence_1, input_sequence_2]

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2],
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

# forward pass
loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
print(loss.item())

generate=model.generate(input_ids=input_ids,attention_mask=attention_mask,max_length=max_target_length)
predicted_text=[tokenizer.decode(g,skip_special_tokens=True,clean_up_tokenization_spaces=True) for g in generate]

for i , pred in enumerate(predicted_text):
    print(f'Input: {input_sequences[i]}')
    print(f'Prediction: {pred}')
    print(f'Actual: {output_sequence_1 if i==0 else output_sequence_2}')

2.038809299468994
Input: New Thing in Town
Prediction: Nouveau héritage en ville
Actual: Comment éliminer les données importantes?
Input: My name is moeed
Prediction: Mon nom est moeed
Actual: Mon nom est moeed
